Modified from original at: http://matgenb.materialsvirtuallab.org/

# Introduction

This notebook demonstrates how you obtain data from the Materials Project using pymatgen's interface to the Materials API. You can find more info about the Materials Project at: https://www.materialsproject.org/

In [1]:
from pymatgen import MPRester, Composition
import re
import pprint

# Make sure that you have a Materials API key. You get that from the Materials Project
# dashboard on their website: https://www.materialsproject.org/dashboard. 
# Once you have your key, put it into the next statement as the argument for MPRester. 
# as MPRester("MY_API_KEY").  You need to put the key in quotes within this call.
mpr = MPRester("0WTRoHhQsJIzTeXr")

# Getting structures with material ids

Let's say you want to find all structures with similar stoichiometry to Fe<sub>2</sub>O<sub>3</sub>. Create a query to the Materials Project:eeee

In [2]:
comp = Composition("Fe2O3")
anon_formula = comp.anonymized_formula
# We need to convert the formula to the dict form used in the database.
anon_formula = {m.group(1): int(m.group(2)) 
                for m in re.finditer(r"([A-Z]+)(\d+)", anon_formula)}

data = mpr.query({"anonymous_formula": anon_formula}, 
                 properties=["task_id", "pretty_formula", "structure"])
print(len(data))  #Should show ~600 data.

610


In [3]:
# data now contains a list of dict. This shows you what each dict has.
# Note that the mp id is named "task_id" in the database itself.
pprint.pprint(data[0])  

{u'pretty_formula': u'Mo2O3',
 u'structure': Structure Summary
Lattice
    abc : 3.3320690399999999 5.642726107748322 15.315099350000001
 angles : 90.0 90.0 107.17273775990958
 volume : 275.11649776015673
      A : 3.3320690399999999 0.0 0.0
      B : -1.66603452 5.39116749 0.0
      C : 0.0 0.0 15.315099350000001
PeriodicSite: Mo (1.6660, 3.9773, 0.7452) [0.8689, 0.7377, 0.0487]
PeriodicSite: Mo (0.0000, 1.4139, 14.5699) [0.1311, 0.2623, 0.9513]
PeriodicSite: Mo (1.6660, 3.9773, 6.9124) [0.8689, 0.7377, 0.4513]
PeriodicSite: Mo (0.0000, 1.4139, 8.4027) [0.1311, 0.2623, 0.5487]
PeriodicSite: Mo (0.0000, 4.1738, 9.7640) [0.3871, 0.7742, 0.6375]
PeriodicSite: Mo (1.6660, 1.2174, 5.5511) [0.6129, 0.2258, 0.3625]
PeriodicSite: Mo (1.6660, 1.2174, 2.1065) [0.6129, 0.2258, 0.1375]
PeriodicSite: Mo (0.0000, 4.1738, 13.2086) [0.3871, 0.7742, 0.8625]
PeriodicSite: O (1.6660, 0.0000, 7.6575) [0.5000, 0.0000, 0.5000]
PeriodicSite: O (1.6660, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
PeriodicSite: 

# Getting band structures

Band structures are fairly large objects. It is not recommended that you download large quantities of bandstructures in one shot, but rather just download the ones you need.

In [4]:
bs = mpr.get_bandstructure_by_material_id("mp-20470")

In [5]:
from pymatgen.electronic_structure.plotter import BSPlotter
%matplotlib inline

In [6]:
plotter = BSPlotter(bs)
plotter.show()

RuntimeError: LaTeX was not able to process the following string:
'lp'
Here is the full report generated by LaTeX: 



RuntimeError: LaTeX was not able to process the following string:
'lp'
Here is the full report generated by LaTeX: 



# Getting elastic constants

We have 5000 elastic constants and growing. You can easily get all the elastic data with materials ids as follows.

In [ ]:
elastic_data = mpr.query({"elasticity": {"$exists": True}}, 
                         properties=["task_id", "pretty_formula", "elasticity"])

In [ ]:
print(len(elastic_data))
pprint.pprint(elastic_data[0])

# More resources

In general, almost any data can be obtained from MP using the MPRester, either via the high-level functions or the very powerful "query" method.

For more complex queries, you can refer to the documentation for the Materials API at https://github.com/materialsproject/mapidoc. 

# Fitting structures

Pymatgen has its own structure matching algorithm, which we have used to effectively reduce the 130,000 structures in ICSD to ~60,000 - 70,000 structures. It is fast and accurate. Here's an example of how it works.

In [ ]:
from pymatgen.analysis.structure_matcher import StructureMatcher

In [ ]:
m = StructureMatcher() # You can customize tolerances etc., but the defaults usually work fine.

In [ ]:
s1 = data[0]["structure"]
print(s1)
s2 = s1.copy()
s2.apply_strain(0.1)
print(s2)

In [ ]:
print(m.fit(s1, s2)) 

For something more challenging, let's see how many structures are similar to Gd2O3

In [ ]:
matches = []
for d in data:
    if m.fit_anonymous(d["structure"], s1):
        matches.append(d)

In [ ]:
# The above fitting took a few seconds. We have 32 similar structures.
print(len(matches))

In [ ]:
# Let's see a few of the matches.
pprint.pprint(matches[0])
pprint.pprint(matches[1])
pprint.pprint(matches[2])

You can see that we have successfully found iso-structural materials!